In [49]:
using Turing

@model gdemo() = begin
    s ~ InverseGamma(2, 3)
    m ~ Normal(0,sqrt(s))
    1.5 ~ Normal(m, sqrt(s))
    2.0 ~ Normal(m, sqrt(s))
    return s, m
  end

# Plain Julia

using ReverseDiff: GradientTape, GradientConfig, gradient, gradient!, compile
using Distributions, DiffBase
using Turing: invlink, logpdf

θ_dim = 2
function lj_func(θ)
  _lj = zero(Real)
  
  d_s = InverseGamma(2, 3)
#   s = invlink(d_s, θ[1])
  s = θ[1]
  _lj += logpdf(d_s, s, false)
  m = θ[2]
  _lj += logpdf(Normal(0, sqrt(s)), m)

  _lj += logpdf(Normal(m, sqrt(s)), 1.5)
  _lj += logpdf(Normal(m, sqrt(s)), 2.0)

  return _lj
end

neg_lj_func(θ) = -lj_func(θ)
const f_tape = GradientTape(neg_lj_func, abs(randn(θ_dim)))
const compiled_f_tape = compile(f_tape)

function grad_func(θ)
    
  inputs = θ
  results = similar(θ)
  all_results = DiffResults.GradientResult(results)

  gradient!(all_results, compiled_f_tape, inputs)

  neg_lj = all_results.value
  grad, = all_results.derivs

  return -neg_lj, grad

end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kai/.julia/v0.6/IJulia/src/execute_request.jl:158
 [5] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kai/.julia/v0.6/Compat/src/Compat.jl:403
 [6] eventloop(::ZMQ.Socket) at /Users/kai/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[49], in expression starting on line 35


grad_func (generic function with 1 method)

In [50]:
mf = gdemo()

[Turing]:  Assume - `s` is a parameter
 @~(::ANY, ::ANY) at compiler.jl:76
[Turing]:  Assume - `m` is a parameter
 @~(::ANY, ::ANY) at compiler.jl:76


##gdemo_model#737 (generic function with 4 methods)

In [51]:
vi = mf()

/=======================================================================
| VarInfo
|-----------------------------------------------------------------------
| Varnames  :   ["{gdemo_var84,s}:1", "{gdemo_var84,m}:1"]
| Range     :   UnitRange{Int64}[1:1, 2:2]
| Vals      :   Real[1.20812, 1.54962]
| RVs       :   Dict{Union{Array{Turing.VarReplay.VarName,1}, Turing.VarReplay.VarName},Any}()
| GIDs      :   [0, 0]
| Orders    :   [0, 0]
| Logp      :   -4.972373307214038
| #produce  :   2
| flags     :   Dict("del"=>Bool[false, false],"trans"=>Bool[false, false])
\=======================================================================


In [57]:
vi.flags["trans"]

2-element Array{Bool,1}:
 false
 false

In [54]:
theta = [1.20812, 1.54962]

2-element Array{Float64,1}:
 1.20812
 1.54962

In [55]:
lj_func(theta)

-4.972377005511801